In [1]:
import numpy as np
import tensorflow as tf
import random

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
def reformat(tens):
    return tens[0].T[0]

In [3]:
def int2bin(i, length=10):
    b = bin(i)[2:].zfill(length)
    b_lst = [int(i) for i in b]
    b_lst.reverse()
    return np.array(b_lst)

def bin2in(s):
    return int(s, 2)

In [4]:
def gen_data():
    max_val = 100
    A = np.random.randint(max_val)
    B = np.random.randint(max_val)
    C = A+B
    Ab = int2bin(A)
    Bb = int2bin(B)
    Cb = int2bin(C)
        
    X_batch = None
    X_batch = np.vstack((Ab, Bb)).T
    X_batch = X_batch[np.newaxis, :]
    y_batch = Cb[np.newaxis, :, np.newaxis]
    
    return X_batch, y_batch

# def gen_data_batch(length):
#     bin_len = 10
    
#     A_lst = [int(i) for i in list(np.random.rand(length)*100)]
#     B_lst = [int(i) for i in list(np.random.rand(length)*100)]
#     C_lst = []
    
#     X = np.zeros((length, bin_len, 2))
#     Y = np.zeros((length, bin_len, 1))
    
#     for idx, (a, b) in enumerate(zip(A_lst, B_lst)):
#         c = a+b
        
#         Ab = int2bin(a, bin_len)
#         Bb = int2bin(b, bin_len)
#         Cb = int2bin(c, bin_len)
        
#         X[idx] = np.vstack((Ab, Bb)).T
#         Y[idx] = Cb[:, np.newaxis]
        
#     return X, Y

In [35]:
def split_train_test(X, Y, ratio=0.7):
    length = X.shape[0]
    
    ran = range(length)
    train_lst = random.sample(ran, int(ratio*length))
    test_lst = list(set(ran)-set(train_lst))
    
    X_train = X[train_lst, :, :]
    Y_train = Y[train_lst, :, :]
    
    X_test = X[test_lst, :, :]
    Y_test = Y[test_lst, :, :]

    return X_train, Y_train, X_test, Y_test

In [6]:
def gen_dataset(length):
    bin_len    = 10

    X = np.zeros((length*length, bin_len,  2))
    Y = np.zeros((length*length, bin_len, 1))    

    idx = 0
    for a in range(length):
        for b in range(length):
            c = a + b

            Ab = int2bin(a, bin_len)
            Bb = int2bin(b, bin_len)
            Cb = int2bin(c, bin_len)

            X[idx] = np.vstack((Ab, Bb)).T
            Y[idx] = Cb[:, np.newaxis]

            idx += 1
    return X, Y

In [7]:
reset_graph()

n_steps   = 10
n_inputs  = 2
n_outputs = 1
n_neurons = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons),
    output_size=n_outputs)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

sm = outputs
# sm = tf.nn.sigmoid(outputs)

init = tf.global_variables_initializer()

In [8]:
# learning_rate = 0.0001
# learning_rate = 0.001
learning_rate = 0.01

loss = tf.reduce_mean(tf.square(sm-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [9]:
X_data, Y_data = gen_dataset(100)

In [36]:
X_train, Y_train, X_test, Y_test = split_train_test(X_data, Y_data)

In [38]:
print(X_train.shape[0])
print(X_test.shape[0])

7000
3000


In [44]:
batch_size   = 50
n_epochs     = 80

saver = tf.train.Saver()

n_examples = X_train.shape[0]

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for iteration in range(n_examples // batch_size):
            X_batch = X_train[iteration*batch_size:(iteration*batch_size)+batch_size,:,:]
            Y_batch = Y_train[iteration*batch_size:(iteration*batch_size)+batch_size,:,:]
        
#             print(X_batch)
#             print(Y_batch)
            sess.run(training_op, feed_dict={X: X_batch, y: Y_batch})

        mse = loss.eval(feed_dict={X: X_batch, y: Y_batch})
        print("\tMSE:", mse)
        
        if mse < 0.001:
            break

    saver.save(sess, "./model")
    
    res = loss.eval(feed_dict={X: X_test, y: Y_test})
    print("test: ", res)

	MSE: 0.167542
	MSE: 0.142198
	MSE: 0.127741
	MSE: 0.113546
	MSE: 0.0826294
	MSE: 0.0631074
	MSE: 0.0490109
	MSE: 0.0401143
	MSE: 0.0325874
	MSE: 0.025678
	MSE: 0.0198152
	MSE: 0.015708
	MSE: 0.0130245
	MSE: 0.0112154
	MSE: 0.00991532
	MSE: 0.00888578
	MSE: 0.00800931
	MSE: 0.00721572
	MSE: 0.0064154
	MSE: 0.00556825
	MSE: 0.00477542
	MSE: 0.00413362
	MSE: 0.00365089
	MSE: 0.00329099
	MSE: 0.00304312
	MSE: 0.00290834
	MSE: 0.00280789
	MSE: 0.00271712
	MSE: 0.00264475
	MSE: 0.00260176
	MSE: 0.00255554
	MSE: 0.00249489
	MSE: 0.00217332
	MSE: 0.00171452
	MSE: 0.00140336
	MSE: 0.00114668
	MSE: 0.000951175
test:  0.000938521


In [12]:
with tf.Session() as sess:
    saver.restore(sess, "./model")
    X_batch, y_batch = gen_data()
    res = sm.eval(feed_dict={X: X_batch, y: y_batch})
#     res = states.eval(feed_dict={X: X_batch, y: y_batch})
#     print(res)
    print(X_batch[0,:,0])
    print(X_batch[0,:,1])
    print(reformat(y_batch))
    
    print(res)
    res[res>0.5] = 1
    res[res<=0.5] = 0
    
    print("prediction errors")
    
    print(reformat(res-y_batch))

INFO:tensorflow:Restoring parameters from ./model
[1 1 0 0 1 1 0 0 0 0]
[0 0 1 1 1 0 1 0 0 0]
[1 1 1 1 0 0 0 1 0 0]
[[[ 0.99898434]
  [ 0.99723959]
  [ 0.99833727]
  [ 1.0055089 ]
  [-0.02557755]
  [ 0.01402092]
  [ 0.06058431]
  [ 0.9470644 ]
  [-0.00574255]
  [ 0.00157905]]]
prediction errors
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
